In [ ]:
from PIL import Image, ImageDraw, ImageOps
import random
from os import system, name
import math
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.transform import hough_line, hough_line_peaks
import copy
from matplotlib.pyplot import figure

In [ ]:
def show_image(data, transpose = False):
    if transpose == True:
        im = Image.fromarray(data.transpose())
    else:
        im = Image.fromarray(data)
    new_im = im.convert('RGB')
    new_im.show()

In [ ]:
def rgb_to_grayscale(pixel):
    result = (0.2126 * pixel[0] + 0.7152 * pixel[1] + 0.0722 * pixel[2])
    return result

In [ ]:
def convert_binary(data, thresh, b):
    if b == 1:
        converted = np.where(data_grayscale < thresh, 255, 0)
    else:
        converted = np.where(data_grayscale < thresh, 0, 255)
    replacements = np.unique(converted, return_counts = True)
    return converted, replacements

In [ ]:
def count_neighbours(pixelsArray, xCoordinate, yCoordinate):
    neighbours = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i != 0 or j != 0:
                neighbours.append(int(pixelsArray[xCoordinate + i][yCoordinate + j] / 255))
                #neighbours.append((xCoordinate + i,yCoordinate + j,int(pixelsSet[xCoordinate + i, yCoordinate + j] / 255)))
    return neighbours

In [ ]:
def follow_line(pixelsArray, startingPixel, width, acceptableLength):
    start = startingPixel[0]
    length = 0 
    intersectionPoints = []
    last10LevelsIncreases = []
    i = 0
    thePixels = []
    
    while startingPixel[0] < width - 1 and -3 < sum(last10LevelsIncreases) < 3:
        toAddLater = []
        
        #print("Now at",startingPixel)
        #print("last10LevelsIncreases is",last10LevelsIncreases)
        #print("")
        thePixels.append(startingPixel)
        # HERE WE LOOK FOR THE CLOSEST PIXEL TO THE RIGHT THAT
        # IS NOT OUTSIDE THE IMAGE
        # IS A BLACK PIXEL
        # IS AT AN ACCEPTABLE DISTANCE 
        i = 1
        while i <= acceptableLength and startingPixel[0] + i < width and pixelsArray[startingPixel[0] + i, startingPixel[1]] != 0:
            toAddLater.append((startingPixel[0] + i, startingPixel[1]))
            i = i + 1
            
        if pixelsArray[startingPixel[0] + i][startingPixel[1]] == 0:
            startingPixel = (startingPixel[0] + i, startingPixel[1])
            thePixels = thePixels + toAddLater
            if len(toAddLater) > 0:
                for point in toAddLater:
                    last10LevelsIncreases.append(0)
                    if len(last10LevelsIncreases) > 10:
                        last10LevelsIncreases = last10LevelsIncreases[1:]
            else:
                last10LevelsIncreases.append(0)
                if len(last10LevelsIncreases) > 10:
                    last10LevelsIncreases = last10LevelsIncreases[1:]
                
            #length = length + 1

        else:
            
            alternatives = []

            # HERE WE CHECK IF THERE IS A BLACK PIXEL ABOVE AND AT THE ACCEPTABLE DISTANCE TO THE RIGHT
            i = 1
            while i <= acceptableLength and pixelsArray[startingPixel[0] + i][startingPixel[1] - 1] != 0:
                i = i + 1
            if pixelsArray[startingPixel[0] + i][startingPixel[1] - 1] == 0 and sum(last10LevelsIncreases) < 2:
                alternatives.append((startingPixel[0] + i, startingPixel[1] - 1, 1))

            # HERE WE CHECK IF THERE IS A BLACK PIXEL BELOW AND AT THE ACCEPTABLE DISTANCE TO THE RIGHT
            i = 1
            while i <= acceptableLength and pixelsArray[startingPixel[0] + i][startingPixel[1] + 1] != 0:
                i = i + 1
            if pixelsArray[startingPixel[0] + i][startingPixel[1] + 1] == 0 and -2 < sum(last10LevelsIncreases):
                alternatives.append((startingPixel[0] + i, startingPixel[1] + 1, -1))
                
            if len(alternatives) > 0:
                
                if len(alternatives) == 2:
                    intersectionPoints.append((startingPixel, alternatives[1], last10LevelsIncreases))
                startingPixel = (alternatives[0][0], alternatives[0][1])
                last10LevelsIncreases.append(alternatives[0][2])
                if len(last10LevelsIncreases) > 10:
                    last10LevelsIncreases = last10LevelsIncreases[1:]

                #length = length + 1
            elif len(intersectionPoints) > 0:
                startingPixel = (intersectionPoints[-1][1][0], intersectionPoints[-1][1][1])
                last10LevelsIncreases.append(intersectionPoints[-1][1][2])
                intersectionPoints = intersectionPoints[0:-1]
                if len(last10LevelsIncreases) > 10:
                    last10LevelsIncreases = last10LevelsIncreases[1:]
            else:
                break
    
    #print("last10LevelsIncreases in the end is",last10LevelsIncreases)
    end = startingPixel[0]
    return thePixels, end - start

In [ ]:
def turning_pixels_white(pixelsArray, height, width, numberOfDarkNeighbours, lr = False):
    turnedWhite = []
    # TURNING ALL PIXELS WITH NO BLACK NEIGHBORS WHITE
    for j in range(1, height - 1):
        for i in range(1, width - 1):
            neighbours = count_neighbours(pixelsArray, i, j)
            # CHECKING IF THE POINT IS BLACK
            if pixelsArray[i][j] == 0:
                # CHECKING IF IT HAS LESS THAN 2 BLACK NEIGHBOURS
                if numberOfDarkNeighbours[0] <= 8 - sum(neighbours) <= numberOfDarkNeighbours[1]:
                #if sum(neighbours) == 8:
                    pixelsArray[i][j] = 255
                    turnedWhite.append((i, j))
                    
                    # IF A PIXEL HAS BEEN TURNED BLACK, WE THEN CHECK IF lr IS TRUE,
                    # IN ORDER TO KNOW IF WE SHOULD CHECK FOR THE LEFT-RIGHT AND UP-DOWN PAIRS
                    # MORE EXACTLY, IF THE PIXELS TO THE LEFT AND TO THE RIGHT (OR ABOVE AND BELOW) ARE BOTH DARK,
                    # THE PIXEL THAT WAS TURNED WHITE HAS TO BE TURNED BACK TO BLACK,
                    # BECAUSE IT IS LIKELY THAT IT WAS PART OF A LINE
                    if lr == True:
                        if (pixelsArray[i - 1][j] == 0 and pixelsArray[i + 1][j] == 0) or (pixelsArray[i][j - 1] == 0 and pixelsArray[i][j + 1] == 0):
                            pixelsArray[i][j] = 0
    return turnedWhite

In [ ]:
def turning_pixels_black(pixelsArray, height, width, lr = False):
    turnedBlack = []
    # TURNING ALL PIXELS WITH MORE THAN 6 BLACK NEIGHBOURS BLACK
    for j in range(1, height - 1):
        for i in range(1, width - 1):
            neighbours = count_neighbours(pixelsArray, i, j)
            # CHECKING IF THE POINT IS WHITE
            if pixelsArray[i][j] == 255:
                if 8 - sum(neighbours) > 6:
                    pixelsArray[i][j] = 0
                    turnedBlack.append((i, j))
                    #turnedBlack = turnedBlack + 1
                # HERE WE CHECK IF THE ABOVE AND BELOW PIXELS ARE BOTH BLACK
                # OR
                # THE PIXELS TO THE LEFT AND TO THE RIGHT ARE BOTH BLACK
                elif lr == True:
                    if neighbours[3] + neighbours[4] == 0 or neighbours[1] + neighbours[6] == 0:
                        pixelsArray[i][j] = 0
                        turnedBlack.append((i, j))
    return turnedBlack

In [ ]:
def count_pixels(pixelsArray, height, width):
    BlackPixels = []
    WhitePixels = []
    for j in range(0, height):
        for i in range(0, width):
            #print(pixelsArray[i][j])
            if pixelsArray[i][j] == 0:
                BlackPixels.append((i, j))
            else:
                WhitePixels.append((i, j))
    return BlackPixels, WhitePixels

In [ ]:
def find_lines(pixelsArray, blackPixelsArray, width, AL):
    startingPoints = []
    heights = list(set([i[1] for i in blackPixelsArray]))
    for i in heights:
        j = 0
        while pixelsArray[j][i] == 255:
            j = j + 1
        startingPoints.append((j, i))

    plPairs = []
    #print('number of starting points:', len(startingPoints))
    for i in startingPoints[0:-1]:
        p, length = follow_line(pixelsArray, (i[0] + int(width / 20), i[1]), width, AL)
        plPairs.append((p, length))
    return startingPoints, plPairs

In [ ]:
def find_region(startingPoint, pixelsArray, commonValue):
    width = pixelsArray.shape[1]
    height = pixelsArray.shape[0]
    verticalStartingPoints = []
    regionPoints = []
    x = startingPoint[1]
    y = startingPoint[0]
    #verticalStartingPoints.append((y, x))
    while x > 0 and pixelsArray[y][x] == commonValue:
        verticalStartingPoints.append((y, x))
        x = x - 1
    if len(verticalStartingPoints) < 2:
        while x < width - 1 and pixelsArray[y][x] == commonValue:
            verticalStartingPoints.append((y, x))
            x = x + 1
            
    for point in verticalStartingPoints:
        x = point[1]
        y = point[0]
        if point[0] == 0:
            while y < width and pixelsArray[y][x] == commonValue:
                regionPoints.append((y, x))
                y = y + 1
        else:
            while y > -1 and pixelsArray[y][x] == commonValue:
                regionPoints.append((y, x))
                y = y - 1
    
    #return verticalStartingPoints
    return regionPoints

In [ ]:
def cluster_lines(lines, N):
    clusters = []
    while len(lines) > 0:
        
        # same WILL CONTAIN LINES THAT SHOULD BE MERGED
        same = [lines[0]]
        
        # ITERATING THROUGH THE SET OF LINES TO SEE IF ANY OF THEM SHOULD BE MERGED WITH A LINE FROM same
        index1 = 0
        while index1 < len(lines):
            
            #print('now at line', index1, 'in the list of lines')
            
            # ITERATING THROUGH THE LIST OF LINES THAT HAVE TO BE MERGED (same)
            index2 = 0
            while index2 < len(same):
                
                #print('now at line', index2, 'in same')
                #print('now going to compare', lines[index1][2], 'with', same[index2][2])
                # CHECKING IF THE STARTING POINT OF TWO LINES DIIFER IN HEIGHT BY LESS THAN N PIXELS
                if lines[index1] != same[index2] and abs(lines[index1][0][int(len(lines[index1][0]) / 2)][1] - same[index2][0][int(len(same[index2][0]) / 2)][1]) < N and lines[index1] not in same:
                    #print('now going to append the line with ID', lines[index1][2])
                    same.append(lines[index1])
                index2 = index2 + 1
            index1 = index1 + 1
            
        if len(same) > 1:
            clusters.append(same)
            
        lines = [i for i in lines if i not in same]
    return clusters

In [ ]:
def average_between_lines(lines):
    #print('I have to merge', len(lines), 'lines')
    if len(lines) > 1:
        averagePoints = []
        points = []
        for line in lines:
            points = points + line[0]
        minX = min(points, key = lambda item: item[0])[0]
        maxX = max(points, key = lambda item: item[0])[0]
        for x in range(minX, maxX):
            pointsX = [i for i in points if i[0] == x]
            if len(pointsX) > 0:
                sumX = sum(i[1] for i in pointsX)
                averagePoints.append((x, int(sumX / len(pointsX))))
        return averagePoints
    else:
        print("THE ARGUMENT OF THE FUNCTION average_between_lines SHOULD BE A LIST OF LENGTH > 1")

In [ ]:
def averageDistance(lines):
    #print(len(lines))
    # INITIALIZING THE EMPTY LIST THAT WILL CONTAIN THE ARRAYS
    # MADE OF DIFFERENCES BETWEEN THE Y COORDINATES OF PIXELS FROM CONSECUTIVE LINES
    distances = []
    
    # INITIALIZING THE EMPTY SET THAT WILL CONTAIN ALL THE COMMON X COORDINATES OF ALL THE LINES IN THE CLUSTER (STAVE)
    commonX = set([])
    
    for line in lines:
        
        # STORING ALL THE X COORDINATES OF THE PIXELS MAKING UP THE LINE
        setX = set([i[0] for i in line])
        
        # WE ONLY KEEP THE X COORDINATES THAT ARE IN ALL LINES IN THE CLUSTER
        if len(commonX) > 0:
            commonX = commonX.intersection(setX)
        else:
            commonX = setX
    
    for i in range(0, len(lines) - 1):

        y1 = np.array([i[1] for i in lines[i] if i[0] in commonX])
        y2 = np.array([i[1] for i in lines[i + 1] if i[0] in commonX])

        distances.append(np.subtract(y2, y1))
    
    arr = np.array(distances, dtype=object)
    
    #print(len(distances))
    
    return commonX, arr.sum(axis = 0) / arr.shape[0]

In [ ]:
def check_if_in_borders(line):
    if min(line, key = lambda item: item[1])[1] > 0 and max(line, key = lambda item: item[1])[1] < height - 1:
        return True
    else:
        return False

In [ ]:
def count_dark_pixels_between_points(pointAbove, pointBelow, maximumBrightness, linesToIgnore = None):
    global pixels2
    count = 0
    
    pixelsToIgnore = []
    for line in linesToIgnore:
        pixelsToIgnore = pixelsToIgnore + line
        
    pixelsInRange = [point[1] for point in pixelsToIgnore if point[0] == pointAbove[0] and pointAbove[1] <= point[1] <= pointBelow[1]]
    
    for i in range(pointAbove[1] + 1, pointBelow[1]):
        if i not in pixelsInRange and sum(pixels2[pointAbove[0], i]) <= maximumBrightness:
                        count = count + 1
    
    return count

In [ ]:
def count_dark_pixels_between_points1(pixelsArray, pointAbove, pointBelow):
    pixelsArray[pointBelow[1]][pointBelow[0]][pointAbove[1]][pointAbove[0]]
    return

## opening the image and converting to an array ##

In [ ]:
imageFile = 'TREE.png'
img = Image.open(imageFile)
pixelsTEST = img.load()
if str(type(pixelsTEST[0, 0])) == "<class 'int'>":
    img = img.convert("RGBA")
data = np.asarray(img)


## viewing the image ##

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111)
ax.set_title(imageFile)
plt.imshow(data)

## converting to grayscale ##

In [ ]:
grayPixelsList = []
for i in range(0, data.shape[0]):
    grayPixels =  [rgb_to_grayscale(pixel) for pixel in data[i]]
    grayPixelsList.append(grayPixels)

data_grayscale = np.array(grayPixelsList)

## viewing the greyscale image ##

In [ ]:
show_image(data_grayscale)

In [ ]:
# saving
im = Image.fromarray(data_grayscale)
new_im = im.convert('RGB')
new_im.save('grayscale.png')

In [ ]:
height = data_grayscale.shape[0]
width = data_grayscale.shape[1]
print('height = ', height)
print('width = ', width)

## calculating the threshold using Otsu's method ##

In [ ]:
thresh = threshold_otsu(data_grayscale)
print('calculated threshold', thresh)

## converting to black and white, based on the calculated threshold ##

In [ ]:
data_binary, count = convert_binary(data_grayscale, thresh, 0)
print('After conversion, there are', count[1][0], 'BLACK pixels and', count[1][1], 'WHITE pixels')

In [ ]:
# saving
im = Image.fromarray(data_binary)
new_im = im.convert('RGB')
new_im.save('black_and_white.png')

## transposing the array, so that columns and rows are switched ##

In [ ]:
print(height, width)
data_binary = data_binary.transpose()

## viewing the image after transposing ##

In [ ]:
show_image(data_binary)

In [ ]:
imageArray = data_binary

In [ ]:
rotatedImageArrayGrayscale = None

## reloading the image  - ONLY RUN AFTER ROTATING THE IMAGE ##

In [ ]:
imageArray = rotatedImageArrayGrayscale

## counting the pixels by colour ##

In [ ]:
result = np.unique(imageArray, return_counts = True)
print(result, result[1][0] + result[1][1])

In [ ]:
BlackPixels, WhitePixels = count_pixels(imageArray, height, width)
#print('There are')
print('black pixels:', len(BlackPixels))
print('white pixels:', len(WhitePixels))
print('Total:', len(BlackPixels) + len(WhitePixels))

## searching for lines, starting at the left-most black pixel of each line *that contains black pixels* ##

In [ ]:
if str(type(rotatedImageArrayGrayscale)) == "<class 'NoneType'>":
    ACCEPTABLE_LENGTH = int(0.01 * width)
else:
    ACCEPTABLE_LENGTH = int(0.005 * width)
startingPoints, HorizontalLines = find_lines(imageArray, BlackPixels, width, ACCEPTABLE_LENGTH)
print('number of starting points:', len(startingPoints))

## here we check how many lines were found ##

In [ ]:
longHorizontalLines = []
minimumLength = int(0.15 * width)
for i in HorizontalLines:
    if i[1] >= minimumLength:
        longHorizontalLines.append(i)
print('number of lines found:', len(HorizontalLines))
print('number of lines longer than the minimum length (', minimumLength, ' pixels): ', len(longHorizontalLines), sep = '')

## converting the array to an image ##

In [ ]:
im = Image.fromarray(imageArray.transpose())
new_im = im.convert('RGB')
img2 = copy.deepcopy(new_im)
d2 = ImageDraw.Draw(img2)
pixels2 = img2.load()

## colouring the starting points and the detected lines with red and blue ##

In [ ]:
for point in startingPoints:
    pixels2[point[0], point[1]] = (255, 0, 0)
for line in longHorizontalLines:
    for point in line[0]:
        pixels2[point[0], point[1]] = (0, 0, 255)

## viewing the image ##

In [ ]:
img2.show()

## detecting the angle of rotation ##

In [ ]:
# here we select the longest line and print its distance
longestLine = max(HorizontalLines, key = lambda item:item[1])
print('the longest line has', longestLine[1], 'pixels')

dx = longestLine[0][-1][0] - longestLine[0][0][0]
dy = longestLine[0][-1][1] - longestLine[0][0][1]
tangent = dy / dx
rotationAngle = math.atan(tangent) * 180 / math.pi
print('tangent:', tangent)
print('angle:', rotationAngle)

## rotating the image by the rotation angle ##

In [ ]:
# TO RUN ONLY IF NOT SATISFIED WITH THE ANGLE
rotatedImage = new_im.rotate(rotationAngle)

## viewing the rotated image ##

In [ ]:
rotatedImage.show()
rotatedImage.save('rotated_1.png')

## converting the rotated image to an array of 0 and 255 ##

In [ ]:
rotatedImageArray = np.array(rotatedImage)

grayPixelsList = []
for i in range(0, rotatedImageArray.shape[0]):
    grayPixels =  [pixel[0] for pixel in rotatedImageArray[i]]
    grayPixelsList.append(grayPixels)

rotatedImageArrayGrayscale = np.array(grayPixelsList)

In [ ]:
width = rotatedImageArrayGrayscale.shape[1]
height = rotatedImageArrayGrayscale.shape[0]

## detecting the dark regions in the four corners, created by the rotation of the image ##

In [ ]:
blackRegions = []
blackRegions.append(find_region((0, width - 1), rotatedImageArrayGrayscale, 0))
blackRegions.append(find_region((0, 0), rotatedImageArrayGrayscale, 0))
blackRegions.append(find_region((height - 1, 0), rotatedImageArrayGrayscale, 0))
blackRegions.append(find_region((height - 1, width - 1), rotatedImageArrayGrayscale, 0))

## turning the points in the dark regions white ##

In [ ]:
for region in blackRegions:
    for point in region:
        rotatedImageArrayGrayscale[point[0]][point[1]] = 255

## viewing the rotated image with the dark regions removed ##

In [ ]:
show_image(rotatedImageArrayGrayscale)

In [ ]:
height = rotatedImageArrayGrayscale.shape[0]
width = rotatedImageArrayGrayscale.shape[1]
print('height = ', height)
print('width = ', width)

In [ ]:
rotatedImageArrayGrayscale = rotatedImageArrayGrayscale.transpose()

In [ ]:
show_image(rotatedImageArrayGrayscale, transpose = True)

In [ ]:
# saving
im = Image.fromarray(rotatedImageArrayGrayscale.transpose())
new_im = im.convert('RGB')
new_im.save('rotated_2.png')

## filtering for lines that are at a distance of at least 5% from the top border ##

In [ ]:
longHorizontalLines = [line for line in longHorizontalLines if line[0][0][1] > 0.05 * height]

## grouping long lines that are very close to each other ##

In [ ]:
clusteredLongHorizontalLines = cluster_lines(longHorizontalLines, 4)

## storing the points separately ##

In [ ]:
longHorizontalLinesPoints = []
for line in longHorizontalLines:
    for point in line[0]:
        longHorizontalLinesPoints.append(point)

## viewing the line count by cluster ##

In [ ]:
%matplotlib inline
clusterSize = [len(cluster) for cluster in clusteredLongHorizontalLines]
x_pos = list(np.arange(0, len(clusterSize), 1))
plt.bar(x_pos, clusterSize)
plt.xticks(x_pos)

plt.gcf().set_size_inches((13, 5))
#plt.figure(figsize=(1, 2)) 

plt.show()

## merging each of the found groups into one line  ##

In [ ]:
mergedLongHorizontalLines = []
index = 0
for cluster in clusteredLongHorizontalLines:
    #print('now at index', index)
    mergedLongHorizontalLines.append(average_between_lines(cluster))
    index = index + 1

In [ ]:
goodLinesPoints = []
for line in mergedLongHorizontalLines:
    goodLinesPoints = goodLinesPoints + [point for point in line]

badLinesPoints = set(longHorizontalLinesPoints) - set(goodLinesPoints)
print(len(badLinesPoints), len(goodLinesPoints))
print('intersection', len(set(badLinesPoints).intersection(goodLinesPoints)))

badLinesHeights = set([point[1] for point in badLinesPoints])
print(len(badLinesHeights))

badLines = []

for h in badLinesHeights:
    badLines.append([point for point in badLinesPoints if point[1] == h])
    
print(len(badLines))

## generating the image and drawing the lines ##

In [ ]:
im = Image.fromarray(imageArray.transpose())
new_im = im.convert('RGB')
img2 = copy.deepcopy(new_im)
d2 = ImageDraw.Draw(img2)
pixels2 = img2.load()

for point in startingPoints:
    pixels2[point[0], point[1]] = (255, 0, 0)
for line in mergedLongHorizontalLines:
    for point in line:
        pixels2[point[0], point[1]] = (0, 0, 255)

## viewing the image ##

In [ ]:
img2.show()
img2.save('lines_3.png')

## grouping the staves ##

In [ ]:
clustersOfFive = []

index = 0
while index + 5 < len(mergedLongHorizontalLines):
    clustersOfFive.append(mergedLongHorizontalLines[index:index + 5])
    index = index + 5

if 5 * len(clustersOfFive) < len(mergedLongHorizontalLines):
    clustersOfFive.append(mergedLongHorizontalLines[index:])

print('found', len([i for i in clustersOfFive if len(i) == 5]), 'clusters with 5 lines')
if len(clustersOfFive[-1]) != 5:
    print('and one cluster with', len(clustersOfFive[-1]))

if len(clustersOfFive[-1]) != 5:
    clustersOfFive = clustersOfFive[0:-1]

staves = []
for group in clustersOfFive:
    gaps = []
    for lineIndex in range(0, len(group) - 1):
        gaps.append(abs(group[lineIndex + 1][0][1] - group[lineIndex][0][1]))
    if max(gaps) / min(gaps) < 3:
        staves.append(group)

clustersOfFive = staves

## generating ledger lines ##

In [ ]:
newClusters = []
ledgerLines = []
distanceBetweenLines = np.array([0])
for cluster in clustersOfFive:
    
    # CHECKING IF THE CLUSTER CONTAINS AT LEAST 2 LINES, SO WE CAN MAKE DETECT A GAP SIZE
    
    # IF THERE IS JUST ONE LINE IN THE CLUSTER, WE USE A PREVIOUS GAP SIZE, IF THERE IS ONE
    # IF THERE ISN'T, WE SKIP THE CLUSTER

    if len(cluster) > 1:
        commonX, distanceBetweenLines = averageDistance(cluster)
        roundedDistanceBetweenLines = np.around(distanceBetweenLines.astype(np.double)).astype(int)
        
    if distanceBetweenLines.all() != None:
        
        firstLine = [i for i in cluster[0] if i[0] in commonX]

        # THE THREE VIRTUAL LINES ABOVE

        ledgerLine1 = []
        for i in range(0, len(firstLine)):
            ledgerLine1.append((firstLine[i][0], firstLine[i][1] - roundedDistanceBetweenLines[i]))
        if check_if_in_borders(ledgerLine1):
            ledgerLines.append(ledgerLine1)

        ledgerLine2 = []
        for point in ledgerLine1:
            ledgerLine2.append((point[0], point[1] - int(np.mean(roundedDistanceBetweenLines))))
        if check_if_in_borders(ledgerLine2):
            ledgerLines.append(ledgerLine2)

        ledgerLine3 = []
        for point in ledgerLine2:
            ledgerLine3.append((point[0], point[1] - int(np.mean(roundedDistanceBetweenLines))))
        if check_if_in_borders(ledgerLine3):
            ledgerLines.append(ledgerLine3)


        lastLine = [i for i in cluster[-1] if i[0] in commonX]

        # THE THREE VIRTUAL LINES BELOW

        ledgerLine4 = []
        for i in range(0, len(lastLine)):
            ledgerLine4.append((lastLine[i][0], lastLine[i][1] + roundedDistanceBetweenLines[i]))
        if check_if_in_borders(ledgerLine4):
            ledgerLines.append(ledgerLine4)

        ledgerLine5 = []
        for point in ledgerLine4:
            ledgerLine5.append((point[0], point[1] + int(np.mean(roundedDistanceBetweenLines))))
        if check_if_in_borders(ledgerLine5):
            ledgerLines.append(ledgerLine5)

        ledgerLine6 = []
        for point in ledgerLine5:
            ledgerLine6.append((point[0], point[1] + int(np.mean(roundedDistanceBetweenLines))))
        if check_if_in_borders(ledgerLine6):
            ledgerLines.append(ledgerLine6)
    
    newCluster = cluster + ledgerLines
    newClusters.append(newCluster)
    ledgerLines = []

## trimming the lines in the clusters so that they all start and end at the same x ##

In [ ]:
trimmedClusters = []
for cluster in newClusters:
    
    # FINDING THE MAXIMUM STARTING X
    # AND THE MINIMUM ENDING X
    maxStart = 0
    minEnd = 1000
    for line in cluster:

        maxi = min(line, key = lambda item: item[0])[0]
        if maxi > maxStart:
            maxStart = maxi

        mini = max(line, key = lambda item: item[0])[0]
        if minEnd < mini:
            minEnd = mini
    
    newCluster = []
    for line in cluster:
        newLine = [point for point in line if maxStart <= point[0] <= minEnd]
        newCluster.append(newLine)
        
    trimmedClusters.append(newCluster)

## sorting the lines in each cluster, by height ##

In [ ]:
sortedClusters = []
for cluster in trimmedClusters:
    sortedClusters.append(sorted(cluster, key = lambda item: item[0][1]))

## adding the ledger lines to the image ##

In [ ]:
for cluster in sortedClusters:
    for line in cluster:
        for point in line:
            pixels2[point[0], point[1]] = (0, 255, 0)

## viewing the image ##

In [ ]:
img2.show()
img2.save('lines_4.png')

## counting the dark pixels between lines ##

In [ ]:
%%time
pixelCountClusters = []
clusterNO = 0
lineNO = 0

for cluster in sortedClusters:
    if len(cluster) > 1:
        
        # A LIST OF COUNTS FOR EACH LINE (EXCEPT THE LAST ONE) IN EACH CLUSTER
        pixelCountCluster = []
        
        # FINDING THE MINIMUM AND MAXIMUM X VALUES FROM THE WHOLE CLUSTER
        startingPoints = []
        endingPoints = []
        for line in cluster:
            minX = min(line, key = lambda item: item[0])[0]
            maxX = max(line, key = lambda item: item[0])[0]
            startingPoints.append(minX)
            endingPoints.append(maxX)
            
        start = max(startingPoints)
        end = min(endingPoints)
        
        lineNO = 0
        for lineIndex in range(0, len(cluster) - 1):
            pixelCount = []
            currentLine = cluster[lineIndex]
            nextLine = cluster[lineIndex + 1]
            for pixel1 in currentLine:
                if start <= pixel1[0] <= end:
                    pixel2 = [pixel for pixel in nextLine if pixel[0] == pixel1[0]]
                    if len(pixel2) > 0:
                        pixel2 = pixel2[0]
                        # ONLY POINTS IN BETWEEN ARE COUNTED
                        lineGap = abs(pixel2[1] - pixel1[1])
                        
                        linesToIgnore = [line for line in badLines if abs(line[0][1] - pixel2[1]) < 3 * lineGap]
                        pixelCount.append((count_dark_pixels_between_points(pixel1, pixel2, 150, linesToIgnore), pixel1, pixel2))
                   
                    else:
                        print("couldn't find a point in line", lineNO, "cluster", clusterNO, "with x equal to", pixel1[0])
            
            pixelCountCluster.append(pixelCount)
            
            lineNO = lineNO + 1
            
        pixelCountClusters.append(pixelCountCluster)
    clusterNO = clusterNO + 1

## finding the minimum and maximum number of dark pixels between all lines ##

In [ ]:
minCount = [1000]
maxCount = [0]
for cluster in pixelCountClusters:
    for line in cluster:
        
        mini = min(line, key = lambda item: item[0])
        if minCount[0] > mini[0]:
            minCount = mini
            
        maxi = max(line, key = lambda item: item[0])
        if maxCount[0] < maxi[0]:
            maxCount = maxi

print('lowest count of dark pixels:', minCount[0])
print('highest count of dark pixels:', maxCount[0])

## visualizing the change in pixel count between lines ##

In [ ]:
yStep = maxCount[0] + 5
canvas = Image.new('RGB', (width + 20, yStep * 16 * len(pixelCountClusters) + 100), "white")

width3, height3 = canvas.size
print(width3, height3)


pixels3 = canvas.load()
c = 1
x = 20
y = 20
for cluster in pixelCountClusters:
    for line in cluster:
        for pixelCount in line:
            try:
                pixels3[x, y - pixelCount[0]] = (0, 0, 0)
            except:
                pass
            x = x + 1
        c = c + 1
        y = y + yStep
        x = 20
    y = y + yStep * 5
        
canvas.show()
canvas.save('pixel_count.png')

## choosing a background ##

In [ ]:
background = minCount[0]
background = 1
print(background)

## reading symbol delimiters ##

In [ ]:
symbolsClusters = []
for cluster in pixelCountClusters:
    symbolsCluster = []
    for gap in cluster:
        index = 0
        symbolsGap = []
        while index < len(gap):
            if gap[index][0] > background:
                start = gap[index]
                while index < len(gap) and gap[index][0] > background:
                    index = index + 1
                if index == len(gap):
                    end = gap[index - 1]
                else:
                    end = gap[index]
                #print(start, end)
                symbolsGap.append((start, end))
            index = index + 1
        symbolsCluster.append(symbolsGap)
    symbolsClusters.append(symbolsCluster)

## viewing the found symbols ##

In [ ]:
im = Image.fromarray(imageArray.transpose())
new_im = im.convert('RGB')
img4 = copy.deepcopy(new_im)
d4 = ImageDraw.Draw(img4)
pixels4 = img4.load()

In [ ]:
for symbolsCluster in symbolsClusters:
    for symbolsGap in symbolsCluster:
        for symbol in symbolsGap:
            d4.rectangle((symbol[0][1][0], symbol[0][1][1], symbol[1][1][0], symbol[1][1][1] + 7), outline = "red")

In [ ]:
img4.show()
img4.save('found_1.png')

In [ ]:
im = Image.fromarray(imageArray.transpose())
new_im = im.convert('RGB')
img4 = copy.deepcopy(new_im)
d4 = ImageDraw.Draw(img4)
pixels4 = img4.load()

## computing the size of the gap between the lines, using the first cluster of lines (staves) ##

In [ ]:
gap = int(np.mean(roundedDistanceBetweenLines))
print(gap)

## choosing a background ##

In [ ]:
sumOfMinimums = 0
for line in pixelCountClusters[0]:
     sumOfMinimums = sumOfMinimums + min([count[0] for count in line])
        
background = int(gap / 2)
print(background)

## reading symbol delimiters ##

In [ ]:
symbolsClusters = []
for cluster in pixelCountClusters:
    symbols = []
    index = 0
    while index < len(cluster[0]):
        
        totalDarkCounts = []
        totalDark = 0
        for line in cluster:
            totalDark = totalDark + line[index][0]
            totalDarkCounts.append(line[index][0])
        
        darkCounts = [totalDarkCounts]
            
        if totalDark > background:
            start = cluster[0][index]
            
            while index < len(cluster[0]) - 1 and totalDark > background:
                darkCounts.append(totalDarkCounts)
                
                index = index + 1
                
                totalDarkCounts = []
                totalDark = 0
                for line in cluster:
                    totalDark = totalDark + line[index][0]
                    totalDarkCounts.append(line[index][0])
                    
                    
                        
            end = cluster[-1][index]
            symbols.append((start, end, darkCounts))
        
        else:
            darkCounts = []
            
        index = index + 1
                    #print(start, end)
            
    symbolsClusters.append(symbols)

In [ ]:
im = Image.fromarray(imageArray.transpose())
new_im = im.convert('RGB')
img4 = copy.deepcopy(new_im)
d4 = ImageDraw.Draw(img4)
pixels4 = img4.load()

## cropping the symbols ##

In [ ]:
import os
# creating a folder for the cropped symbols
folders = os.listdir()
if 'crop' not in folders:
    os.mkdir('crop')
stave = 1
for cluster in symbolsClusters:
    id = 1
    for symbol in cluster:
        if abs(symbol[0][1][0] - symbol[1][1][0]) > 1.3 * gap:
            d4.rectangle((symbol[0][1][0], symbol[0][1][1], symbol[1][1][0], symbol[1][1][1]), outline = "red")
            imCrop = img4.crop((symbol[0][1][0] - gap, symbol[0][1][1] - gap, symbol[1][1][0] + gap, symbol[1][1][1] + gap))
            imCrop.save("crop/" + "stave_" + str(stave) + "_" + str(id) + ".png")
            id = id + 1
    stave = stave + 1

In [ ]:
img4.show()
img4.save('found_2.png')